<a href="https://colab.research.google.com/github/rajlm10/Touchstone-Foundation/blob/main/TSF_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TSF Foundation Assessment - Raj Sangani** <br>
*My approach :*<br>


1.   To find the features that really matter, I will use backward elimination with a significance level of 5 percent.
2.   I will use the statsmodels api to fit least squares regression and get the P values for every fit.
3. To explain how well variables define the demands, I will again use the optimal model's coefficients to see the correlation.
4. I will use the model with only the neccessary features to predict on a test set from 'day.csv' **Please note that test.csv on kaggle does not contain the prediction column hence I had to use day.csv for testing as well.**
5. I will evaluate using the *R squared and Adjusted R squared metric.*






In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dataset = pd.read_csv('day.csv')

In [3]:
dataset.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


As we can see the dataset has many variables which may be redundant and the last column is what we are interested in predicting, the count value. I will omit the dteday column as the year and month columns are also given separately.

In [4]:
X=dataset.iloc[:,2:-1].values # Construct the X np matrix using all but first two and last columns
X.shape

(499, 13)

In [6]:
y=dataset.iloc[:,-1].values #Construct the y matrix using the last (count) column
y.shape

(499,)

Now I will prepare the data for backward elimination by appending a column of ones (bias term of equation) to the X matrix


In [9]:
X = np.append(arr = np.ones((X.shape[0],1)).astype(int), values = X, axis = 1) 
X.shape

(499, 14)

I will use the **OLS** (Ordinary Least Squares) regression to find redundant features and build the model

Initially I fit the data with all columns and as we can see all the **P values** are listed below which will be helpful while determining feature importance

In [10]:
X_opt = X[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.190e+31
Date:                Sat, 17 Apr 2021   Prob (F-statistic):               0.00
Time:                        16:29:02   Log-Likelihood:                 12826.
No. Observations:                 499   AIC:                        -2.562e+04
Df Residuals:                     485   BIC:                        -2.557e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.853e-12   5.78e-13     -4.938      0.000   -3.99e-12   -1.72e-12
x1          1.064e-12   1.49e-13      7.152      0.000    7.71e-13    1.36e-12
x2          3.155e-12   3.27e-13      9.646      0.000    2.51e-12     3.8e-12
x3          2.895e-13   4.58e-14      6.328      0.000       2e-13    3.79e-13
x4          1.137e-13   4.82e-13      0.236      0.813   -8.33e-13    1.06e-12
x5          1.227e-13   3.84e-14      3.198      0.001    4.73e-14    1.98e-13
x6          3.411e-13   2.88e-13      1.183      0.238   -2.26e-13    9.08e-13
x7         -3.588e-13   1.89e-13     -1.903      0.058   -7.29e-13    1.18e-14
x8          1.101e-13   1.29e-13      0.855      0.393   -1.43e-13    3.63e-13
x9         -1.084e-13   1.19e-13     -0.910      0.363   -3.42e-13    1.26e-13
x10          6.55e-15      7e-15      0.936      0.350   -7.21e-15    2.03e-14
x11        -3.086e-14   1.65e-14     -1.875      0.061   -6.32e-14    1.48e-15
x12            1.0000   2.25e-16   4.44e+15      0.000       1.000       1.000
x13            1.0000   1.53e-16   6.52e+15      0.000       1.000       1.000
==============================================================================
Omnibus:                       25.167   Durbin-Watson:                   0.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.483
Skew:                          -0.433   Prob(JB):                     2.16e-05
Kurtosis:                       2.467   Cond. No.                     2.66e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.66e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Now I define the backward elimination


1.   Define a significance level for features (I chose 5 percent)
2.   Fit the model with all features and start removing the feature with maximum P value if that P value is greater than our significance level

3.  Now remove that column and fit the model with the reduced set of features

4. Repeat until no such P values exist

5. We now have our optimal features



In [11]:
def backwardElimination(x, significance_level):
    num_features = len(x[0])
    for i in range(0, num_features):
        regressor_OLS = sm.OLS(y, x).fit()
        max_P_Value = max(regressor_OLS.pvalues).astype(float)
        if max_P_Value > significance_level:
            for j in range(0, num_features - i):
                if (regressor_OLS.pvalues[j].astype(float) == max_P_Value):
                    x = np.delete(x, j, 1)
    regressor_OLS.summary()
    return x

In [12]:
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13]]
X_Modeled = backwardElimination(X_opt, SL)


As you can see below we get the optimal features and also the R-squared and Adjusted R-squared as 1 which is very promising

In [13]:
regressor_OLS = sm.OLS(endog=y, exog=X_Modeled).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.152e+31
Date:                Sat, 17 Apr 2021   Prob (F-statistic):               0.00
Time:                        16:34:20   Log-Likelihood:                 12831.
No. Observations:                 499   AIC:                        -2.565e+04
Df Residuals:                     492   BIC:                        -2.562e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.958e-12   3.35e-13     -5.850      0.000   -2.62e-12    -1.3e-12
x1           8.58e-13   1.43e-13      6.018      0.000    5.78e-13    1.14e-12
x2          3.382e-12   2.43e-13     13.947      0.000    2.91e-12    3.86e-12
x3          2.132e-13   4.42e-14      4.821      0.000    1.26e-13       3e-13
x4         -3.353e-14   1.47e-14     -2.280      0.023   -6.24e-14   -4.64e-15
x5             1.0000   1.33e-16   7.49e+15      0.000       1.000       1.000
x6             1.0000    8.6e-17   1.16e+16      0.000       1.000       1.000
==============================================================================
Omnibus:                      220.274   Durbin-Watson:                   0.098
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.669
Skew:                           0.173   Prob(JB):                     3.61e-07
Kurtosis:                       1.857   Cond. No.                     1.52e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.52e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Now we will compare the original columns with our optimal ones to see which features were selected**

In [14]:
pd.DataFrame(np.append(arr = np.ones((X.shape[0],1)).astype(int), values = X, axis = 1)[:10])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,2.0,14.110847,18.18125,80.5833,10.749882,331.0,654.0
1,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,14.902598,17.68695,69.6087,16.652113,131.0,670.0
2,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,8.050924,9.47025,43.7273,16.636703,120.0,1229.0
3,1.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,8.200000,10.60610,59.0435,10.739832,108.0,1454.0
4,1.0,1.0,1.0,0.0,1.0,0.0,3.0,1.0,1.0,9.305237,11.46350,43.6957,12.522300,82.0,1518.0
5,1.0,1.0,1.0,0.0,1.0,0.0,4.0,1.0,1.0,8.378268,11.66045,51.8261,6.000868,88.0,1518.0
6,1.0,1.0,1.0,0.0,1.0,0.0,5.0,1.0,2.0,8.057402,10.44195,49.8696,11.304642,148.0,1362.0
7,1.0,1.0,1.0,0.0,1.0,0.0,6.0,0.0,2.0,6.765000,8.11270,53.5833,17.875868,68.0,891.0
8,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5.671653,5.80875,43.4167,24.250650,54.0,768.0
9,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,6.184153,7.54440,48.2917,14.958889,41.0,1280.0


In [15]:
pd.DataFrame(X_Modeled[:10])

,0,1,2,3,4,5,6
0,1.0,1.0,0.0,1.0,10.749882,331.0,654.0
1,1.0,1.0,0.0,1.0,16.652113,131.0,670.0
2,1.0,1.0,0.0,1.0,16.636703,120.0,1229.0
3,1.0,1.0,0.0,1.0,10.739832,108.0,1454.0
4,1.0,1.0,0.0,1.0,12.522300,82.0,1518.0
5,1.0,1.0,0.0,1.0,6.000868,88.0,1518.0
6,1.0,1.0,0.0,1.0,11.304642,148.0,1362.0
7,1.0,1.0,0.0,1.0,17.875868,68.0,891.0
8,1.0,1.0,0.0,1.0,24.250650,54.0,768.0
9,1.0,1.0,0.0,1.0,14.958889,41.0,1280.0


As we can see columns  **season , yr , month ,windspeed,casual and registered matter** the most and are enough to build a model (leaving aside the first bias column)

We can also see from the coefficients that all but the **windspeed** column are positively correlated.
*The casual and registered column hold alot of meaning and have large coefficients per unit value of the columns.*

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_Modeled[:,1:], y, test_size = 0.2, random_state = 0) #Splitting by 80 percent-20 percent
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(399, 6)
(100, 6)
(399,)
(100,)


Fitting the regressor using optimal features. Note that the normalize attribute is set to true and hence we need not scale and centre the data manually

In [18]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.normalize=True 
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

**Using the R2 score metric as said in the question**

In [21]:
y_pred=regressor.predict(X_test)
y_pred.shape

(100,)

In [22]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

1.0

**Also verifying the mean squared error**

In [23]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)



1.1696850849382657e-24

***As we can see both metrics are very good the R squared is 1.0 and MSE is so close to 0.***

Additionally I will also predict on the test.csv file although the cnt column is not available in this kaggle dataset so if you have a test column ready please verify the initial values

In [24]:
additional_ds = pd.read_csv('test.csv')
additional_ds.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,501,16-05-2019,2,1,5,0,3,1,1,26.103347,29.79875,69.7917,8.208304,991,6433
1,502,17-05-2019,2,1,5,0,4,1,1,24.326653,28.63065,52.0000,15.374825,1242,6142
2,503,18-05-2019,2,1,5,0,5,1,1,23.130847,27.55605,52.3333,9.166739,1521,6118
3,504,19-05-2019,2,1,5,0,6,0,1,24.600000,28.34540,45.6250,5.626325,3410,4884
4,505,20-05-2019,2,1,5,0,0,0,1,25.454153,29.19835,53.0417,17.042589,2704,4425


In [27]:
#As we see the cnt column is missing, nevertheless I will predict it using our model
X_additional=additional_ds.iloc[:,[2,3,4,12,13,14]].values
X_additional.shape

(230, 6)

In [28]:
y_predicted_additional=regressor.predict(X_additional)
y_predicted_additional.shape

(230,)

In [29]:
pd.DataFrame(y_predicted_additional[:20]) #First 20 values

,0
0,7424.0
1,7384.0
2,7639.0
3,8294.0
4,7129.0
5,4359.0
6,6073.0
7,5260.0
8,6770.0
9,6734.0
